# Map Page

> The page of the app that generates the map view of the data.  

In [ ]:
#| default_exp pages/map_population

In [ ]:
#| export
# TODO minimise these imports a bit where possible
from nbdev.showdoc import *
# TODO work out how to get around below hack
try:
    from SolomonIslandsDataMap.dash_components import gen_pyramid, gen_dd, gen_kpi
    from SolomonIslandsDataMap.app_data import map_graph, year_slider\
        , popPyramid, popKpi, mytitle
    from SolomonIslandsDataMap.load_data import SolomonGeo
except: 
    from dash_components import gen_pyramid, gen_dd, gen_kpi
    from app_data import  map_graph, year_slider\
        , popPyramid,  popKpi, mytitle
    from load_data import SolomonGeo
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio # Unless this is used graphs will not be dynamic?
import numpy as np
from fastcore.test import *
from dash import Dash, callback, dcc, Output, Input, State, html, Patch, ctx, register_page  # pip install dash
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components
from dash_bootstrap_templates import load_figure_template
import dash_mantine_components as dmc
import os
import json

In [ ]:
#| export
# Try catch is added here so that the notebook can be tested without the app being run
try:
    register_page(__name__, 
                        path='/population_projections',
                        title='Population Projections',
                        name='Population Projections')
except:
    pass

This data store is used to trigger callbacks on page load and to know which page is loaded.

In [ ]:
#| export
# this initial data store is used to trigger callbacks on page load and know
init_init = dcc.Store(id="initial-initial", data='pop')


Setup some of  the key data contiaining objects

## Customize the Layout

In [ ]:
#| export
def layout():
    return  init_init, mytitle, map_graph, year_slider,\
        dbc.Row([
                popKpi,
                dbc.Col([popPyramid], width = 8, align = 'center')
            ], justify = 'around'), 

### Some things to note about this page
- On Load, we change geo to province if it isn't alread there,
- We can just use the normal geo drowpdown because it is set to and will remain as Province, the other dropdown is basically just for display, 
- As a result, we can recycle a lot of the other dropdowns,
- Probably the only other one we will overwrite is the map one, to instead display population data. 
- Also we add the age bracket dropdown
- And we use a different measure and variable selection
- It's really just updated to display new data not change to much of the other variables and callbacks


# Build callbacks to create interaction

## Update the variable dropdowns based on the selection of the overall variable

In [ ]:
#| export
@callback(
    Output('measurePopDiv', 'children', allow_duplicate=True),
    Input('varDropdownPop', 'value'),
    State('geo_df', 'data'),
    State('measureDropdownPop', 'value'),
    allow_duplicate=True,
    prevent_initial_call=True
)
def update_measure_pop(new_var:str, # Selected variable
                   dict_sol:dict, # The dataset in dictionary form
                   measure:str, # Currently selected measure
              )->dcc.Dropdown: # Returns a dropdown of measures for selected variable
    '''
    Updates the dropdown_location dropdown based on the currently selected data aggregation.
    '''
    sol_geo = SolomonGeo.gen_stored(dict_sol) # reload the data

    # Sometimes this callback is triggered when the measure doesn't need to be reset.
    # Check whether measure is in variable, if not reset to 0
    if measure not in sol_geo.population_vars[new_var]:
        measure = sol_geo.population_vars[new_var][0]

    # When a variable is selected, the measure will be set as the first one
    return gen_dd(sol_geo.population_vars[new_var], 'measureDropdownPop', 
                  val = measure)

In [ ]:
#| hide
show_doc(update_measure_pop)

---

### update_measure_pop

>      update_measure_pop (new_var:str, dict_sol:dict, measure:str)

Updates the dropdown_location dropdown based on the currently selected data aggregation.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| new_var | str | Selected variable |
| dict_sol | dict | The dataset in dictionary form |
| measure | str | Currently selected measure |
| **Returns** | **Dropdown** | **Returns a dropdown of measures for selected variable** |

## Update the measure selection from Population Pyramid clicks

In [ ]:
#| export
# TODO add the population pyramid clicks one here. Needs population pyramid, use below markdown for inspiration
        


#| export
@callback(
    Output('measureDropdown', 'value', allow_duplicate=True),
    Output(selectedBarGraph, "clickData"),
    Input(selectedBarGraph, 'clickData'),
    State('varDropdown', 'value'),
    State('geo_df', 'data'),
    prevent_initial_call=True,
    allow_duplicate=True,
)
def bar_click(clickData:dict, # The currently clicked location on bar graph
                variable:str, # The currently selected variable
                dict_sol:dict, # The dataset in dictionary form
                )->[str]: # Returns the new value for the dropdown
    """This function updates the dropdown menu based on the bar graph click data"""
    print("func bc")
    sol_geo = SolomonGeo.gen_stored(dict_sol) # reload the data
    if clickData is None:
        print("Click data was none")
        return sol_geo.census_vars[variable][0], None
    else:
        # The measure are list of dictionaries
        selection = list(map(lambda x: x['x'], clickData['points']))[0]
    
        # returned objects are assigned to the component property of the Output
        # After updating fileter, we always reset map selection 
        return selection, None
        

In [ ]:
#| hide
#show_doc(bar_click)

### Update map based on inputs

In [ ]:
# TODO should this be a function related to the other map scirpt?? 

In [ ]:
#| export
# TODO - this could quite liekly be in map_page calbback as well
# TODO - add age here
@callback(
    Output('map', 'figure', allow_duplicate=True),
    Input("segmented_geo", 'value'),
    Input("segmented_type", 'value'),
    Input('measureDropdownPop', 'value'),
    Input('varDropdownPop', 'value'),
    Input('age_dropdown', 'value'),
    Input("year_slider", "value"),
    Input('segmented_geo', 'disabled'),
    State('geo_df', 'data'),
    allow_duplicate=True,
    prevent_initial_call=True)
def update_map_pop(geog:str, # current geography
                    data_type:str, # User input of type of data
                    measure:str, # A string contiaining the census variable and measure split by ':'
                    variable:str, # The state of the variable dropdown
                    age:[str], # Age Brackets to display
                    year:int, # The selected projection year
                    geo_trigger:int, # Listening for whether segmented geo is locked, signaling that map
                                        # page was loaded after geo data was updated.
                    dict_sol:dict, # The dataset in dictionary form
              )->type(go.Figure()): # Returns a graph object figure after being updated and the dynamic title
    '''
    Updates the focus census variable or geography dispalayed on the map.
    Geo_trigger listens for when the geography is blacked out. This happens when the population map page is loaded,
    in particular, it happens after the geography variable is fixed. In this case, we re update everything on the map
    '''
    # TODO the None workaround might be taxing on the load times, is there a better way
    # or maybe I can check it it needs updating?
    patched_figure = Patch()
    button_clicked = ctx.triggered_id
    init_load = False
    if geo_trigger == True: init_load = True

    sol_geo = SolomonGeo.gen_stored(dict_sol) # reload the data

    # A None value is passed when the page is first loaded, hence
    # the the values are reset.
    # Hardcoded to province as we only have forcasts by province
    if button_clicked in ["segmented_geo"] or init_load == True:
        # Update disaplayed geography 
        for geo in sol_geo.geo_levels:
            tn = np.where(sol_geo.geo_levels == geo)[0][0] # Tracks the trace number
            patched_figure['data'][tn]['visible'] = geog == geo
        
    if button_clicked in ["segmented_type"] or init_load == True:
        # Update the type of data displayed on map and the hover template
        for geo in sol_geo.geo_levels:
            tn = np.where(sol_geo.geo_levels == geo)[0][0] # Tracks the trace number
            # All years allows us to set the min and max colour as the min and max across all years
            all_years = sol_geo.get_pop(years = sol_geo.pop_years, var = variable, measure = measure, type_filter=data_type,
                                  agg = True, agg_location = True, ages = age)
            ar = all_years.loc[year].values[:, -1]
            all_years = all_years.values[:, -1]
            ar = ar.reshape((ar.shape[0],))
            if data_type == 'Total':
                ht = '%{customdata} <extra>%{z}</extra>'
            elif data_type == 'Proportion':
                ht = '%{customdata} <extra>%{z:.1%}</extra>'
            else:
                ValueError("Data type of map not recognised and note accounted for")
            patched_figure['data'][tn]['z'] = ar
            patched_figure['data'][tn]['zmin'] = np.min(all_years)
            patched_figure['data'][tn]['zmax'] = np.max(all_years)
            patched_figure['data'][tn]['hovertemplate'] = ht

            
        

    if button_clicked in ['measureDropdownPop', 'year_slider', 'age_dropdown'] or init_load == True:
        # Update the z values in map to the data for the requested census variable
        for geo in sol_geo.geo_levels:
        # Ar updates the z value ie. data disaplyed each time
        # TODO this is fairly inefficient, as we are processing each time
        # Maybe faster framework like polars could help? or caching but would require a lot of caching
            tn = np.where(sol_geo.geo_levels == geo)[0][0] # Tracks the trace number
            
            # All years allows us to set the min and max colour as the min and max across all years
            all_years = sol_geo.get_pop(years = sol_geo.pop_years, var = variable, measure = measure, type_filter=data_type,
                                  agg = True, agg_location = True, ages = age)
            ar = all_years.loc[year].values[:, -1]
            all_years = all_years.values[:, -1]
            ar = ar.reshape((ar.shape[0],))
            patched_figure['data'][tn]['z'] = ar
            patched_figure['data'][tn]['zmin'] = np.min(all_years)
            patched_figure['data'][tn]['zmax'] = np.max(all_years)
        
    # returned objects are assigned to the component property of the Output
    # After updating fileter, we always reset map selection 

    return patched_figure

In [ ]:
#| hide
show_doc(update_map_pop)

---

### update_map_pop

>      update_map_pop (geog:str, data_type:str, measure:str, variable:str,
>                      age:[<class'str'>], year:int, geo_trigger:int,
>                      dict_sol:dict)

Updates the focus census variable or geography dispalayed on the map.
Geo_trigger listens for when the geography is blacked out. This happens when the population map page is loaded,
in particular, it happens after the geography variable is fixed. In this case, we re update everything on the map

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| geog | str | current geography |
| data_type | str | User input of type of data |
| measure | str | A string contiaining the census variable and measure split by ':' |
| variable | str | The state of the variable dropdown |
| age | [<class 'str'>] | Age Brackets to display |
| year | int | The selected projection year |
| geo_trigger | int | Listening for whether segmented geo is locked, signaling that map |
| dict_sol | dict | The dataset in dictionary form |
| **Returns** | **Figure** | **Returns a graph object figure after being updated and the dynamic title** |

### Update the selected part of the map based on the dropdown

### Population Pyramid

In [ ]:
#| export
# Callback allows components to interact
@callback(
    Output('popPyramid', 'figure'),
    Input("segmented_type", 'value'),
    Input('measureDropdownPop', 'value'),
    Input('locDropdown', 'value'),
    Input('age_dropdown', 'value'),
    Input("year_slider", "value"),
    State("segmented_geo", 'value'),
    State('varDropdownPop', 'value'),
    State('geo_df', 'data'),
    allow_duplicate=True,
    prevent_initial_call=True
)
def update_pyramid(data_type:str, # User input of type of data
                     measure:str, # A string contiaining the census variable and measure split by ':'
                     loc_selection:[str], # The selected locations, may be none
                     ages:[str], # Currently selected locations for highlighting
                     year:str, # Year of projection data
                     geo_input:str, # User input from the geography dropdown
                     variable:str, # The state of the variable dropdown
                     dict_sol:dict, # The dataset in dictionary form
                     #init_trigger:str, # uncessary variable inside the callback
              )->(type(go.Figure()), str): # Returns a graph object figure after being updated and the dynamic title
    '''
    Updates the focus census variable or geography dispalayed on the map
    '''
    sol_geo = SolomonGeo.gen_stored(dict_sol) # reload the data
    fig = gen_pyramid(sol_geo = sol_geo, geo_filter = geo_input, year = year, 
                      variable = variable, locations = loc_selection, type_filter = data_type, ages = ages)
    
    return fig

On click, update the contents of the kpi's

In [ ]:
#| hide
show_doc(update_pyramid)

---

### update_pyramid

>      update_pyramid (data_type:str, measure:str, loc_selection:[<class'str'>],
>                      ages:[<class'str'>], year:str, geo_input:str,
>                      variable:str, dict_sol:dict)

Updates the focus census variable or geography dispalayed on the map

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| data_type | str | User input of type of data |
| measure | str | A string contiaining the census variable and measure split by ':' |
| loc_selection | [<class 'str'>] | The selected locations, may be none |
| ages | [<class 'str'>] | Currently selected locations for highlighting |
| year | str | Year of projection data |
| geo_input | str | User input from the geography dropdown |
| variable | str | The state of the variable dropdown |
| dict_sol | dict | The dataset in dictionary form |
| **Returns** | **(<class 'plotly.graph_objs._figure.Figure'>, <class 'str'>)** | **Returns a graph object figure after being updated and the dynamic title** |

## Update Kpi
Based on Current Selection

In [ ]:
#| export
# Callback allows components to interact
@callback(
    Output('popKpi', 'children'),
    Input("segmented_type", 'value'),
    Input('measureDropdownPop', 'value'),
    Input('locDropdown', 'value'),
    Input('age_dropdown', 'value'),
    Input("year_slider", "value"),
    State('varDropdownPop', 'value'),
    State('geo_df', 'data'),
    allow_duplicate=True,
    prevent_initial_call=True
)
def update_kpi(data_type:str, # User input of type of data
                     measure:str, # A string contiaining the census variable and measure split by ':'
                     loc_selection:[str], # The selected locations, may be none
                     ages:[str], # Currently selected locations for highlighting
                     year:str, # Year of projection data
                     variable:str, # The state of the variable dropdown
                     dict_sol:dict, # The dataset in dictionary form
              )->(dcc.Markdown, dcc.Markdown): # Returns a graph object figure after being updated and the dynamic title
    '''
    Updates the focus census variable or geography dispalayed on the map
    '''
    sol_geo = SolomonGeo.gen_stored(dict_sol) # reload the data    

    return gen_kpi(sol_geo, year, variable, measure, ages, loc = loc_selection, type_filter = data_type)

In [ ]:
show_doc(update_kpi)

---

### update_kpi

>      update_kpi (data_type:str, measure:str, loc_selection:[<class'str'>],
>                  ages:[<class'str'>], year:str, variable:str, dict_sol:dict)

Updates the focus census variable or geography dispalayed on the map

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| data_type | str | User input of type of data |
| measure | str | A string contiaining the census variable and measure split by ':' |
| loc_selection | [<class 'str'>] | The selected locations, may be none |
| ages | [<class 'str'>] | Currently selected locations for highlighting |
| year | str | Year of projection data |
| variable | str | The state of the variable dropdown |
| dict_sol | dict | The dataset in dictionary form |
| **Returns** | **(<class 'dash.dcc.Markdown.Markdown'>, <class 'dash.dcc.Markdown.Markdown'>)** | **Returns a graph object figure after being updated and the dynamic title** |

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()